In [1]:
# 1) First: install Java, Spark and and run a local Spark session by just running this on Google Colab:

!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   # !apt-get --> install java
!wget -q https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz  # !wget  --> download file from url
!tar xf spark-3.1.3-bin-hadoop3.2.tgz  # !tar --> like unzip 
!pip install -q findspark  # !pip  --> instal a package, we cant import a library without installing it first, most libraries that we used were already installed

# This are INSTALLATION COMMANDS IN LINUX that we run in our collab space, it's similar to downloading programs an installing them on our computers
# installs Apache Spark 3.1.2, Java 8, and Findspark, a library that makes it easy for Python to find Spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [824 kB]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# 2) Second: set the locations where Spark and Java are installed to let know Colab where to find it.

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [3]:
# 3) Third: import spark libraries and use them

import findspark
findspark.init("spark-3.1.3-bin-hadoop3.2") # SPARK_HOME

from pyspark.sql import SparkSession

# Create the session - We need to remember to close it at the end
# The session is basically our connection to Spark layer in the Hadoop ecosystem
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import Row
from pyspark.sql import functions as F

In [9]:
# Code for connecting our google drive to this collab notebook
from google.colab import drive
drive.mount('/content/drive')

# After you finish, look at the menu (gui to the left) and check under "files" that your "drive" folder was added.

Mounted at /content/drive


In [10]:
!ls "/content/drive/MyDrive/spark"  # Check what files we have in the path
# Notice! we have "u.item" file in that folder

 au.csv			   de_state_cities.csv	      users2.csv
 books_df.csv		   in.csv		      users3.csv
 BX-Book-Ratings.csv	   morecolforbooks.csv	      users_df.csv
 BXBooks.csv		   ratings_df.csv	      users_file1.csv
 bxbookswithnewcols.csv    Spark.ipynb		      users_file2.csv
 BX-Users.csv		  'Spark - Questions.ipynb'   users_file3.csv
 canadacities.csv	   u.data		      users_file.csv
 city_country_dict.csv	   u.item		     'users_to finish_work.csv'
 country_city_dict.csv	   uk_cities.csv	      us-states-territories.csv
 Cust_Rev_Very_Short.csv   uscities.csv		      walmart_stock.csv
 data			   users1.csv


In [11]:
# reading csv file to dataframe
spark = SparkSession.builder.appName("bxbooks").getOrCreate()
books = spark.read.options(delimiter=';') \
  .csv("/content/drive/MyDrive/spark/BXBooks.csv",header=True,inferSchema=True)
books.printSchema()


root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Image-URL-S: string (nullable = true)
 |-- Image-URL-M: string (nullable = true)
 |-- Image-URL-L: string (nullable = true)



In [12]:
books = books.drop("Image-URL-S","Image-URL-M","Image-URL-L")

In [13]:
books.columns

['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']

In [14]:
books.show(10)

+----------+--------------------+--------------------+-------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|
+----------+--------------------+--------------------+-------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|               1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|               1999|W. W. Norton &amp...|
|0399135782|The Kitchen God's...|             Amy Tan|               1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|               2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|       

<br>
<br>
<hr class="dashed">
<br>
<br>

In [15]:
books.dtypes

[('ISBN', 'string'),
 ('Book-Title', 'string'),
 ('Book-Author', 'string'),
 ('Year-Of-Publication', 'int'),
 ('Publisher', 'string')]

In [18]:
newColumns = ["isbn","booktitele","bookauthor","year_of_publication", "publisher"]
books =books.toDF(*newColumns)
books.show(20)

+----------+--------------------+--------------------+-------------------+--------------------+
|      isbn|          booktitele|          bookauthor|year_of_publication|           publisher|
+----------+--------------------+--------------------+-------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|               1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|               1999|W. W. Norton &amp...|
|0399135782|The Kitchen God's...|             Amy Tan|               1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|               2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|       

In [11]:
books.describe().show()

+-------+--------------------+--------------------+--------------+-------------------+--------------------+
|summary|                isbn|          booktitele|    bookauthor|year_of_publication|           publisher|
+-------+--------------------+--------------------+--------------+-------------------+--------------------+
|  count|              271379|              271379|        271379|             271379|              271379|
|   mean|1.0412353852604752E9|            Infinity|          null| 1959.7560496574902|  25032.333333333332|
| stddev|1.4877523499740572E9|                 NaN|          null| 258.01136256381125|   21676.03850645531|
|    min|          0000913154| A Light in the S...|      D. Chiel|                  0| Editions P. Terrail|
|    max|          B0002K6K8O|   �?�?thique en toc|�?�?ric Holder|               2050|   �?�?ditions 10/18|
+-------+--------------------+--------------------+--------------+-------------------+--------------------+



In [12]:
import requests

In [16]:
# geting the categories from google books in the internet
def get_category(isbn):
  try:
    url = 'https://www.googleapis.com/books/v1/volumes?q=isbn:' + isbn
    response = requests.get(url)
    data = response.json()
    category = data['items'][0]['volumeInfo']['categories'][0]    
    pageCount = data['items'][0]['volumeInfo']['pageCount']
    language = data['items'][0]['volumeInfo']['language']
  except: 
    category,pageCount,language= 'other',0, 'unknown'
  return isbn,category,pageCount,language
      

In [17]:
get_category('0425054594')

('0425054594',
 'other',
 0,
 'unknown',
 {'items': [{'accessInfo': {'accessViewStatus': 'NONE',
     'country': 'US',
     'embeddable': False,
     'epub': {'isAvailable': False},
     'pdf': {'isAvailable': False},
     'publicDomain': False,
     'quoteSharingAllowed': False,
     'textToSpeechPermission': 'ALLOWED',
     'viewability': 'NO_PAGES',
     'webReaderLink': 'http://play.google.com/books/reader?id=EU1bAAAAMAAJ&hl=&printsec=frontcover&source=gbs_api'},
    'etag': 'WQ9CrTin0tY',
    'id': 'EU1bAAAAMAAJ',
    'kind': 'books#volume',
    'saleInfo': {'country': 'US',
     'isEbook': False,
     'saleability': 'NOT_FOR_SALE'},
    'selfLink': 'https://www.googleapis.com/books/v1/volumes/EU1bAAAAMAAJ',
    'volumeInfo': {'allowAnonLogging': False,
     'authors': ['E. E. Smith'],
     'averageRating': 2.5,
     'canonicalVolumeLink': 'https://books.google.com/books/about/Galactic_Patrol.html?hl=&id=EU1bAAAAMAAJ',
     'contentVersion': '0.2.1.0.preview.0',
     'imageLinks':

In [ ]:
from pyspark.sql import functions  as F

In [ ]:
list_isbn =books.select('isbn').rdd.flatMap(lambda x: x).collect()

In [ ]:
len(list_isbn)

271379

In [ ]:
list_isbn

In [ ]:
list = []

In [ ]:
import time
list = list
a =10000
b = 15000
while a < 272000:
  for i in list_isbn[a:b]:
    list = list + [get_category(i)]
    time.sleep(0.6)
  a= a+5000
  b= b+5000
  time.sleep(10)
  print(a)


15000
20000


In [ ]:
len(list)

In [ ]:
list[9500:10000]

[('0440222842', 'Fiction', 304, 'en'),
 ('0373272383', 'Fiction', 250, 'en'),
 ('0373194714', 'Fiction', 192, 'en'),
 ('0399141227', 'Fiction', 448, 'en'),
 ('3426619342', 'Communication and sex', 345, 'de'),
 ('3426624680', 'other', 0, 'unknown'),
 ('3426273098', 'other', 0, 'unknown'),
 ('3810505153', 'other', 0, 'unknown'),
 ('3499134039', 'Cats', 281, 'de'),
 ('3518370081', 'other', 0, 'unknown'),
 ('3548359698', 'Communication in marriage', 396, 'de'),
 ('0553379011', 'Fiction', 325, 'en'),
 ('8090128920', 'other', 0, 'unknown'),
 ('8476021844', 'other', 0, 'unknown'),
 ('0385319851', 'Fiction', 207, 'en'),
 ('0060958308', 'Fiction', 246, 'en'),
 ('0440418321', 'Juvenile Fiction', 399, 'en'),
 ('0316789976', 'Fiction', 246, 'en'),
 ('0394525280', 'Fiction', 288, 'en'),
 ('0020433115', 'Friendship Juvenile fiction', 128, 'en'),
 ('0671733397', "Children's stories", 117, 'en'),
 ('8423310728', 'Fiction', 130, 'es'),
 ('1568360428', 'Self-Help', 133, 'en'),
 ('0671519816', 'Fiction',

In [ ]:
cols = ["isbn1","category","pageCount","language"]
df1 = spark.createDataFrame(list,schema= cols)

In [ ]:
df1.show()

+----------+--------+---------+--------+
|     isbn1|category|pageCount|language|
+----------+--------+---------+--------+
|0195153448|   other|        0| unknown|
|0002005018|   other|        0| unknown|
|0060973129|   other|        0| unknown|
|0374157065| Medical|      330|      en|
|0393045218|   other|        0| unknown|
|0399135782| Fiction|      415|      en|
|0425176428| History|      395|      en|
|0671870432|   other|        0| unknown|
|0679425608|   other|        0| unknown|
|074322678X|   other|        0| unknown|
|0771074670| Fiction|      225|      en|
|080652121X|   other|        0| unknown|
|0887841740|   other|        0| unknown|
|1552041778|   other|        0| unknown|
|1558746218|   other|        0| unknown|
|1567407781|   other|        0| unknown|
|1575663937|   other|        0| unknown|
|1881320189|   other|        0| unknown|
|0440234743|   other|        0| unknown|
|0452264464|   other|        0| unknown|
+----------+--------+---------+--------+
only showing top

In [ ]:
df1.groupBy("category").count().orderBy("count",ascending=False).show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|             Fiction|  257|
|               other|  251|
|    Juvenile Fiction|   35|
|Biography & Autob...|   21|
|            Religion|    9|
|             History|    7|
|   Political Science|    6|
|      Social Science|    6|
| Body, Mind & Spirit|    6|
|Business & Economics|    5|
|               Humor|    5|
|              Poetry|    4|
| Juvenile Nonfiction|    4|
|  Literary Criticism|    3|
|Language Arts & D...|    3|
|          Philosophy|    3|
|Family & Relation...|    3|
|    Health & Fitness|    2|
|  Children's stories|    2|
|          Psychology|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
books = books.join( df1, books.isbn == df1.isbn1, 'left')

In [ ]:
books = books.drop("isbn1")

In [ ]:
books.show()

In [ ]:
books.coalesce(1).write.option("header" ,"True").csv("/content/drive/MyDrive/spark/bxbookswithnewcols1.csv","overwrite",header =True)

In [ ]:
# Stop the session --> similar to saving our doings in spark layer
spark.stop()

<br>
<br>
<hr class="dashed">
<br>
<br>